In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
## Check
def _(DataSet):
    lst_fold = list(set(DataSet["fold"]))
    lst_fold.sort()
    for j in range(len(lst_fold)):
        df_classes = DataSet[DataSet["fold"]==lst_fold[j]]
        Class_folder = list(set(df_classes["image_folder"]))
        lst_folder = list(set(df_classes["fold"]))
        print(f"[INFO]: [{j+1}].  Fold {lst_fold[j]} With Shape ==> {len(Class_folder)} Folder, {df_classes.shape}")
        print("Check 6-Fold With Class:")
        lst_cls = list(set(df_classes['subclass']))
        print(f"Number of Classes {len(lst_cls)} Class ==> {lst_cls}")
        for m in lst_cls:
            df_f =  df_classes[df_classes["subclass"]==m]
            list_folder = list(set(df_f["image_folder"]))
            list_code =  list(set(df_f["Code"]))
            print(f"[INFO]:  Classse {m} ==> With {len(list_folder)} Folder, {df_f.shape[0]} images!! ==> {len(list_code)} PerSon!!")
        print("="*100)
    
    return

In [3]:
def mergeDatarame(df_2dFFT, df_ML) :
    df_outer = pd.merge(df_2dFFT, df_ML, on='Code', how='outer')
    print(df_outer.shape)
    df_outer_notNAN = df_outer[df_outer['fold'].notna()]
    df_outer_notNAN["fold"] = df_outer_notNAN["fold"].astype('int')
    print("After Merge DataFrame", df_outer_notNAN.shape)
    df_outer_notNAN_ = df_outer_notNAN[df_outer_notNAN['MCV'].notna()]
    print(df_outer_notNAN_.shape)
    df_outer_notNAN_ = df_outer_notNAN_.reset_index(drop=True)
    
    return df_outer_notNAN_

## 📌 ใช้ค่า MCV, MCH, Hb เทรน ML เพื่อแยก thalas โดยยึด Fold ตามแบบเดิม

- 🩸แบบ 2 โรค (HN, NBL)  ==> 6 Fold

- 🩸แบบ 3 โรค (Normal, ตัดม้าม, ไม่ตัดม้าม) ==> 3 Fold

### 📂1. DATA : ค่าจากผล Lab

In [4]:
df_dictCode = pd.read_csv("/home/kannika/code/Rheology2023/Rheology_Blood/All_lab_MCV-MCH-Hb_Fold_NotNAN.csv")
print(df_dictCode.shape)
## NotNaN
df_dictCode_ = df_dictCode[df_dictCode['fold'].notna()]
df_dictCode_["fold"] = df_dictCode_["fold"].astype('int')
print(df_dictCode_.shape)
df_dictCode_.head()

(64, 7)
(51, 7)


/tmp/ipykernel_10704/53047531.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dictCode_["fold"] = df_dictCode_["fold"].astype('int')


,Unnamed: 0,Code,MCV,MCH,Hb,sheet,fold
0,0,HN01,75.8,23.1,6.0,Splenectomy,6
1,1,HN02,73.5,22.4,7.2,Splenectomy,3
6,6,HN12,75.4,24.1,9.8,Splenectomy,1
7,7,HN13,72.6,23.4,5.8,Splenectomy,1
8,8,HN16,76.0,24.7,10.6,Splenectomy,5


In [5]:
df_ML = df_dictCode[['Code', 'MCV', 'MCH', 'Hb']]
df_ML

,Code,MCV,MCH,Hb
0,HN01,75.8,23.1,6.0
1,HN02,73.5,22.4,7.2
2,HN03,77.3,24.3,9.0
3,HN04,79.8,25.6,7.6
4,HN06,78.5,25.9,8.2
...,...,...,...,...
59,NBL19,89.5,27.8,14.3
60,NBL20,95.3,31.7,14.7
61,NBL21,74.9,23.9,14.5
62,NBL22,75.1,24.7,15.2


In [ ]:
## เหลือ 51 Row

> ### 6 Fold

In [6]:
### ✅ 2 Class => (NBL, HN)
DF_A = pd.read_csv('/home/kannika/code/Rheology2023/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-2dFFTdataset-3channels-6Fold-splitclass.csv')
print(DF_A.shape)
print("Fold:", set(DF_A["fold"]))
DF_A.head()

(288, 15)
Fold: {1, 2, 3, 4, 5, 6}


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary
0,0,0,0,237,237,HN35_D0_30HZ_20XINF_UWELL_25660131_162433_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162433,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1
1,1,1,1,234,234,HN35_D0_30HZ_20XINF_UWELL_25660131_161754_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_161754,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1
2,2,2,2,324,324,HN43_D0_30HZ_20XINF_UWELL_25660330_153259_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN43_D0_30HZ_20XINF_UWELL_25660330_153259,SSD_Backup,HN,HN43,No_Splenectomy,1,"[2, 5, 7]",1
3,3,3,3,268,268,HN38_D0_30HZ_20XINF_UWELL_25660223_145146_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN38_D0_30HZ_20XINF_UWELL_25660223_145146,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1
4,4,4,4,239,239,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1


In [23]:
DF_A_res = mergeDatarame(DF_A, df_ML)
print("Fold: ", set(DF_A_res["fold"]))
DF_A_res.head()

(301, 18)
After Merge DataFrame (288, 18)
(268, 18)
Fold:  {1, 2, 3, 4, 5, 6}


/tmp/ipykernel_10704/1519000798.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_outer_notNAN["fold"] = df_outer_notNAN["fold"].astype('int')


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,0.0,0.0,0.0,237.0,237.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162433_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162433,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
1,1.0,1.0,1.0,234.0,234.0,HN35_D0_30HZ_20XINF_UWELL_25660131_161754_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_161754,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
2,4.0,4.0,4.0,239.0,239.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
3,8.0,8.0,8.0,240.0,240.0,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
4,2.0,2.0,2.0,324.0,324.0,HN43_D0_30HZ_20XINF_UWELL_25660330_153259_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN43_D0_30HZ_20XINF_UWELL_25660330_153259,SSD_Backup,HN,HN43,No_Splenectomy,1,"[2, 5, 7]",1.0,57.4,17.7,4.4


In [8]:
fold_res = DF_A_res[DF_A_res['fold']!= 1].reset_index(drop=True)
print(fold_res.shape)
fold_res.head()

(220, 18)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,48.0,48.0,48.0,232.0,232.0,HN34_D0_30HZ_20XINF_UWELL_25660131_161340_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN34_D0_30HZ_20XINF_UWELL_25660131_161340,SSD_Backup,HN,HN34,No_Splenectomy,2,"[2, 5, 7]",1.0,57.6,18.3,9.3
1,50.0,50.0,50.0,228.0,228.0,HN34_D0_30HZ_20XINF_UWELL_25660131_160441_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN34_D0_30HZ_20XINF_UWELL_25660131_160441,SSD_Backup,HN,HN34,No_Splenectomy,2,"[2, 5, 7]",1.0,57.6,18.3,9.3
2,54.0,54.0,54.0,231.0,231.0,HN34_D0_30HZ_20XINF_UWELL_25660131_161127_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN34_D0_30HZ_20XINF_UWELL_25660131_161127,SSD_Backup,HN,HN34,No_Splenectomy,2,"[2, 5, 7]",1.0,57.6,18.3,9.3
3,56.0,56.0,56.0,227.0,227.0,HN34_D0_30HZ_20XINF_UWELL_25660131_160236_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN34_D0_30HZ_20XINF_UWELL_25660131_160236,SSD_Backup,HN,HN34,No_Splenectomy,2,"[2, 5, 7]",1.0,57.6,18.3,9.3
4,59.0,59.0,59.0,230.0,230.0,HN34_D0_30HZ_20XINF_UWELL_25660131_160914_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN34_D0_30HZ_20XINF_UWELL_25660131_160914,SSD_Backup,HN,HN34,No_Splenectomy,2,"[2, 5, 7]",1.0,57.6,18.3,9.3


In [9]:
fold_res_Normal = fold_res[fold_res['subclass']=="Normal"].reset_index(drop=True)
print("Fold: ", set(fold_res_Normal["fold"]))
print(fold_res_Normal.shape)
fold_res_Normal.head()

Fold:  {2, 3, 4, 5, 6}
(100, 18)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,62.0,62.0,62.0,487.0,487.0,NBL19_D0_30HZ_20XINF_UWELL_25660117_142019_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL19_D0_30HZ_20XINF_UWELL_25660117_142019,SSD_Backup,NBL,NBL19,Normal,2,"[2, 5, 7]",0.0,89.5,27.8,14.3
1,69.0,69.0,69.0,491.0,491.0,NBL19_D0_30HZ_20XINF_UWELL_25660117_142906_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL19_D0_30HZ_20XINF_UWELL_25660117_142906,SSD_Backup,NBL,NBL19,Normal,2,"[2, 5, 7]",0.0,89.5,27.8,14.3
2,74.0,74.0,74.0,488.0,488.0,NBL19_D0_30HZ_20XINF_UWELL_25660117_142230_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL19_D0_30HZ_20XINF_UWELL_25660117_142230,SSD_Backup,NBL,NBL19,Normal,2,"[2, 5, 7]",0.0,89.5,27.8,14.3
3,76.0,76.0,76.0,490.0,490.0,NBL19_D0_30HZ_20XINF_UWELL_25660117_142657_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL19_D0_30HZ_20XINF_UWELL_25660117_142657,SSD_Backup,NBL,NBL19,Normal,2,"[2, 5, 7]",0.0,89.5,27.8,14.3
4,108.0,108.0,108.0,504.0,504.0,NBL20_D0_30HZ_20XINF_UWELL_25660119_144352_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL20_D0_30HZ_20XINF_UWELL_25660119_144352,SSD_Backup,NBL,NBL20,Normal,3,"[2, 5, 7]",0.0,95.3,31.7,14.7


In [10]:
set_code = list(set(fold_res_Normal["Code"]))
set_code.sort()
print("Number of Patient :", len(set_code))
print("#"*120)
#set_code
len_img = []
for i in range(len(set_code)):
    DF_code = fold_res_Normal[fold_res_Normal["Code"]==set_code[i]]
    print(f"[INFO]: Patient({i+1}.)  For Code Number : {set_code[i]} ==> {DF_code.shape[0]} images ")
    len_img.append(DF_code.shape[0])
    print("="*120)

Number of Patient : 13
########################################################################################################################
[INFO]: Patient(1.)  For Code Number : NBL08 ==> 10 images 
[INFO]: Patient(2.)  For Code Number : NBL09 ==> 8 images 
[INFO]: Patient(3.)  For Code Number : NBL10 ==> 6 images 
[INFO]: Patient(4.)  For Code Number : NBL11 ==> 9 images 
[INFO]: Patient(5.)  For Code Number : NBL12 ==> 7 images 
[INFO]: Patient(6.)  For Code Number : NBL13 ==> 8 images 
[INFO]: Patient(7.)  For Code Number : NBL16 ==> 8 images 
[INFO]: Patient(8.)  For Code Number : NBL17 ==> 8 images 
[INFO]: Patient(9.)  For Code Number : NBL18 ==> 8 images 
[INFO]: Patient(10.)  For Code Number : NBL19 ==> 4 images 
[INFO]: Patient(11.)  For Code Number : NBL20 ==> 11 images 
[INFO]: Patient(12.)  For Code Number : NBL21 ==> 5 images 
[INFO]: Patient(13.)  For Code Number : NBL22 ==> 8 images 


- Fold ละ 2 คน
- เศษ 3 คน ==> 4, 5, 6 ภาพ

8, 8, 8, 8, 8

Res==> [4,5,6,7,8,9,10,11]




In [11]:
'''
10, 11, 9+4, 8+5,6+7
|
|
10, 11, 13, 13, 13
                  +
8,  8,  8,  8 , 8
                  =
18, 19, 21, 21, 21
|
|
|
2,  2,  3,  3,  3  ==> Patient

--------------------------------
- fold-2 ==> (NBL08 + NBL13) 
- fold-3 ==> (NBL16 + NBL20)
- fold-4 ==> (NBL11 + NBL17 + NBL19)
- fold-5 ==> (NBL09 + NBL18 + NBL21)
- fold-6 ==> (NBL10 + NBL12 + NBL22)
'''

'\n10, 11, 9+4, 8+5,6+7\n|\n|\n10, 11, 13, 13, 13\n                  +\n8,  8,  8,  8 , 8\n                  =\n18, 19, 21, 21, 21\n|\n|\n|\n2,  2,  3,  3,  3  ==> Patient\n\n--------------------------------\n- fold-2 ==> (NBL08 + NBL13) \n- fold-3 ==> (NBL16 + NBL20)\n- fold-4 ==> (NBL11 + NBL17 + NBL19)\n- fold-5 ==> (NBL09 + NBL18 + NBL21)\n- fold-6 ==> (NBL10 + NBL12 + NBL22)\n18,    19,    21,    21,    21\n'

In [21]:
## Create Dict  for Map Fold
## 1. Create Dict
output_dict = {'NBL08': 2, 'NBL13': 2, 'NBL16': 3, 'NBL20': 3, 'NBL11': 4, 'NBL17': 4, 'NBL19': 4, 'NBL09': 5,'NBL18': 5,
                   'NBL21': 5, 'NBL10': 6,'NBL12': 6,'NBL22': 6}  
print(output_dict)

{'NBL08': 2, 'NBL13': 2, 'NBL16': 3, 'NBL20': 3, 'NBL11': 4, 'NBL17': 4, 'NBL19': 4, 'NBL09': 5, 'NBL18': 5, 'NBL21': 5, 'NBL10': 6, 'NBL12': 6, 'NBL22': 6}


In [24]:
## 2. Map fold
fold_res_Normal["fold"] = fold_res_Normal['Code'].map(output_dict)
print(fold_res_Normal.shape)
fold_res_Normal.head()

(100, 18)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,62.0,62.0,62.0,487.0,487.0,NBL19_D0_30HZ_20XINF_UWELL_25660117_142019_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL19_D0_30HZ_20XINF_UWELL_25660117_142019,SSD_Backup,NBL,NBL19,Normal,4,"[2, 5, 7]",0.0,89.5,27.8,14.3
1,69.0,69.0,69.0,491.0,491.0,NBL19_D0_30HZ_20XINF_UWELL_25660117_142906_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL19_D0_30HZ_20XINF_UWELL_25660117_142906,SSD_Backup,NBL,NBL19,Normal,4,"[2, 5, 7]",0.0,89.5,27.8,14.3
2,74.0,74.0,74.0,488.0,488.0,NBL19_D0_30HZ_20XINF_UWELL_25660117_142230_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL19_D0_30HZ_20XINF_UWELL_25660117_142230,SSD_Backup,NBL,NBL19,Normal,4,"[2, 5, 7]",0.0,89.5,27.8,14.3
3,76.0,76.0,76.0,490.0,490.0,NBL19_D0_30HZ_20XINF_UWELL_25660117_142657_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL19_D0_30HZ_20XINF_UWELL_25660117_142657,SSD_Backup,NBL,NBL19,Normal,4,"[2, 5, 7]",0.0,89.5,27.8,14.3
4,108.0,108.0,108.0,504.0,504.0,NBL20_D0_30HZ_20XINF_UWELL_25660119_144352_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL20_D0_30HZ_20XINF_UWELL_25660119_144352,SSD_Backup,NBL,NBL20,Normal,3,"[2, 5, 7]",0.0,95.3,31.7,14.7


In [25]:
image_path = fold_res_Normal["image_path"].tolist()
print(len(image_path))
image_fold = fold_res_Normal["fold"].tolist()
print(len(image_fold))

100
100


In [27]:
for i in range(len(DF_A_res)):
    path_A = DF_A_res["image_path"][i]
    for j in range(len(image_path)):
        path_B = image_path[j]
        if path_A == path_B : 
            DF_A_res.loc[DF_A_res.index[i], "fold"] = image_fold[j]

print(DF_A_res.shape)
DF_A_res.head()

(268, 18)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,0.0,0.0,0.0,237.0,237.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162433_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162433,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
1,1.0,1.0,1.0,234.0,234.0,HN35_D0_30HZ_20XINF_UWELL_25660131_161754_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_161754,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
2,4.0,4.0,4.0,239.0,239.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
3,8.0,8.0,8.0,240.0,240.0,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
4,2.0,2.0,2.0,324.0,324.0,HN43_D0_30HZ_20XINF_UWELL_25660330_153259_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN43_D0_30HZ_20XINF_UWELL_25660330_153259,SSD_Backup,HN,HN43,No_Splenectomy,1,"[2, 5, 7]",1.0,57.4,17.7,4.4


In [28]:
DF_A_res.isnull().values.any()

False

In [29]:
list(set(DF_A_res["fold"]))

[1, 2, 3, 4, 5, 6]

In [30]:
_(DF_A_res)

[INFO]: [1].  Fold 1 With Shape ==> 48 Folder, (48, 18)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 12 Folder, 12 images!! ==> 3 PerSon!!
[INFO]:  Classse Normal ==> With 24 Folder, 24 images!! ==> 3 PerSon!!
[INFO]:  Classse No_Splenectomy ==> With 12 Folder, 12 images!! ==> 3 PerSon!!
[INFO]: [2].  Fold 2 With Shape ==> 42 Folder, (42, 18)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 12 Folder, 12 images!! ==> 3 PerSon!!
[INFO]:  Classse Normal ==> With 18 Folder, 18 images!! ==> 2 PerSon!!
[INFO]:  Classse No_Splenectomy ==> With 12 Folder, 12 images!! ==> 3 PerSon!!
[INFO]: [3].  Fold 3 With Shape ==> 43 Folder, (43, 18)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 12 Folder, 12 images!! ==> 3 PerSon!!
[INFO]:

In [31]:
## Save to CSV. 
DF_A_res.to_csv('/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_2dFFTdataset_3channels_6Fold_split2class-ver2.csv')

----------------------------------------------------------------------------------------------------------------

> ### 3 Fold

In [32]:
### ✅ 3 Class => (Normal, ตัดม้าม, ไม่ตัดม้าม)
DF_B = pd.read_csv('/home/kannika/code/Rheology2023/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-2dFFTdataset-3channels-3Fold-split3class.csv')
print(DF_B.shape)
print("Fold:", set(DF_B["fold"]))
DF_B.head()

(288, 15)
Fold: {1, 2, 3}


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary
0,0,0,0,239,239,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1
1,1,1,1,269,269,HN38_D0_30HZ_20XINF_UWELL_25660223_145353_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN38_D0_30HZ_20XINF_UWELL_25660223_145353,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1
2,2,2,2,240,240,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1
3,3,3,3,265,265,HN38_D0_30HZ_20XINF_UWELL_25660223_144217_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN38_D0_30HZ_20XINF_UWELL_25660223_144217,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1
4,4,4,4,266,266,HN38_D0_30HZ_20XINF_UWELL_25660223_144746_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN38_D0_30HZ_20XINF_UWELL_25660223_144746,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1


In [33]:
DF_B_res = mergeDatarame(DF_B, df_ML)
print("Fold: ", set(DF_B_res["fold"]))
DF_B_res.head()

(301, 18)
After Merge DataFrame (288, 18)
(274, 18)
Fold:  {1, 2, 3}


/tmp/ipykernel_10704/1519000798.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_outer_notNAN["fold"] = df_outer_notNAN["fold"].astype('int')


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,0.0,0.0,0.0,239.0,239.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
1,2.0,2.0,2.0,240.0,240.0,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
2,5.0,5.0,5.0,234.0,234.0,HN35_D0_30HZ_20XINF_UWELL_25660131_161754_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_161754,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
3,14.0,14.0,14.0,236.0,236.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162205_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162205,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
4,15.0,15.0,15.0,238.0,238.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162641_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162641,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1


In [34]:
_(DF_B_res)

[INFO]: [1].  Fold 1 With Shape ==> 82 Folder, (82, 18)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 32 Folder, 32 images!! ==> 6 PerSon!!
[INFO]:  Classse Normal ==> With 18 Folder, 18 images!! ==> 4 PerSon!!
[INFO]:  Classse No_Splenectomy ==> With 32 Folder, 32 images!! ==> 6 PerSon!!
[INFO]: [2].  Fold 2 With Shape ==> 96 Folder, (96, 18)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 32 Folder, 32 images!! ==> 6 PerSon!!
[INFO]:  Classse Normal ==> With 32 Folder, 32 images!! ==> 6 PerSon!!
[INFO]:  Classse No_Splenectomy ==> With 32 Folder, 32 images!! ==> 6 PerSon!!
[INFO]: [3].  Fold 3 With Shape ==> 96 Folder, (96, 18)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 32 Folder, 32 images!! ==> 6 PerSon!!
[INFO]:

In [35]:
DF_B_res.isnull().values.any()

False

In [36]:
DF_B_res.to_csv('/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_2dFFTdataset_3channels_3Fold_split3class-ver2.csv')

In [16]:
fold_res_B = DF_B_res[DF_B_res['fold']!= 1].reset_index(drop=True)
print(fold_res_B.shape)

fold_resB_Normal = fold_res_B[fold_res_B['subclass']=="Normal"].reset_index(drop=True)
print("Fold: ", set(fold_resB_Normal["fold"]))
print(fold_resB_Normal.shape)
fold_resB_Normal.head()

(192, 18)
Fold:  {2, 3}
(64, 18)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,112.0,112.0,112.0,462.0,462.0,NBL16_D0_30HZ_20XINF_UWELL_25651227_142145_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL16_D0_30HZ_20XINF_UWELL_25651227_142145,SSD_Backup,NBL,NBL16,Normal,2,"[2, 5, 7]",0.0,79.6,24.9,13.2
1,114.0,114.0,114.0,458.0,458.0,NBL16_D0_30HZ_20XINF_UWELL_25651227_141159_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL16_D0_30HZ_20XINF_UWELL_25651227_141159,SSD_Backup,NBL,NBL16,Normal,2,"[2, 5, 7]",0.0,79.6,24.9,13.2
2,120.0,120.0,120.0,461.0,461.0,NBL16_D0_30HZ_20XINF_UWELL_25651227_141932_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL16_D0_30HZ_20XINF_UWELL_25651227_141932,SSD_Backup,NBL,NBL16,Normal,2,"[2, 5, 7]",0.0,79.6,24.9,13.2
3,121.0,121.0,121.0,465.0,465.0,NBL16_D0_30HZ_20XINF_UWELL_25651227_142909_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL16_D0_30HZ_20XINF_UWELL_25651227_142909,SSD_Backup,NBL,NBL16,Normal,2,"[2, 5, 7]",0.0,79.6,24.9,13.2
4,123.0,123.0,123.0,456.0,456.0,NBL16_D0_30HZ_20XINF_UWELL_25651227_140659_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL16_D0_30HZ_20XINF_UWELL_25651227_140659,SSD_Backup,NBL,NBL16,Normal,2,"[2, 5, 7]",0.0,79.6,24.9,13.2


In [17]:
set_code = list(set(fold_resB_Normal["Code"]))
set_code.sort()
print("Number of Patient :", len(set_code))
print("#"*120)
#set_code
len_img = []
for i in range(len(set_code)):
    DF_code = fold_resB_Normal[fold_resB_Normal["Code"]==set_code[i]]
    print(f"[INFO]: Patient({i+1}.)  For Code Number : {set_code[i]} ==> {DF_code.shape[0]} images ")
    len_img.append(DF_code.shape[0])
    print("="*120)

Number of Patient : 12
########################################################################################################################
[INFO]: Patient(1.)  For Code Number : NBL08 ==> 9 images 
[INFO]: Patient(2.)  For Code Number : NBL09 ==> 5 images 
[INFO]: Patient(3.)  For Code Number : NBL10 ==> 4 images 
[INFO]: Patient(4.)  For Code Number : NBL11 ==> 6 images 
[INFO]: Patient(5.)  For Code Number : NBL12 ==> 3 images 
[INFO]: Patient(6.)  For Code Number : NBL13 ==> 3 images 
[INFO]: Patient(7.)  For Code Number : NBL16 ==> 6 images 
[INFO]: Patient(8.)  For Code Number : NBL17 ==> 6 images 
[INFO]: Patient(9.)  For Code Number : NBL18 ==> 5 images 
[INFO]: Patient(10.)  For Code Number : NBL20 ==> 6 images 
[INFO]: Patient(11.)  For Code Number : NBL21 ==> 4 images 
[INFO]: Patient(12.)  For Code Number : NBL22 ==> 7 images 


In [18]:
patien_NBL01 = fold_resB_Normal[fold_resB_Normal["Code"]== 'NBL01']
print(patien_NBL01.shape)
patien_NBL01

(0, 18)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb


In [19]:
patien_NBL00 = fold_resB_Normal[fold_resB_Normal["Code"]== 'NBL00']
print(patien_NBL00.shape)
patien_NBL00

(0, 18)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb


##  Map 6-Fold and 3-Fold With ML Dataset

In [37]:
feture_ML = df_dictCode_[['Code', 'MCV', 'MCH', 'Hb']]
print(feture_ML.shape)
feture_ML.head()

(51, 4)


,Code,MCV,MCH,Hb
0,HN01,75.8,23.1,6.0
1,HN02,73.5,22.4,7.2
6,HN12,75.4,24.1,9.8
7,HN13,72.6,23.4,5.8
8,HN16,76.0,24.7,10.6


In [38]:
print(len(feture_ML["Code"]))
print(len(list(set(feture_ML["Code"]))))

51
51


> ### 6- Fold: 2 Classes       

In [49]:
DF_A_res_ML = DF_A_res.drop_duplicates(subset='Code', keep="last")
DF_A_res_ML_ = DF_A_res_ML[["Code", "classes", "subclass", "fold", "classes_binary", "MCV", "MCH", "Hb"]]
DF_A_res_ML_.reset_index(drop=True)
print(DF_A_res_ML_.shape)
DF_A_res_ML_

(51, 8)


,Code,classes,subclass,fold,classes_binary,MCV,MCH,Hb
3,HN35,HN,No_Splenectomy,1,1.0,74.3,23.4,7.1
6,HN43,HN,No_Splenectomy,1,1.0,57.4,17.7,4.4
11,HN38,HN,No_Splenectomy,1,1.0,72.9,22.6,8.0
17,NBL14,NBL,Normal,1,0.0,86.8,28.1,13.2
26,NBL15,NBL,Normal,1,0.0,88.2,27.9,13.9
35,NBL23,NBL,Normal,1,0.0,66.5,20.3,13.0
40,HN12,HN,Splenectomy,1,1.0,75.4,24.1,9.8
43,HN13,HN,Splenectomy,1,1.0,72.6,23.4,5.8
47,HN17,HN,Splenectomy,1,1.0,75.0,25.0,7.1
52,HN34,HN,No_Splenectomy,2,1.0,57.6,18.3,9.3


In [52]:
print(set(DF_A_res_ML_["fold"]))
print(set(DF_A_res_ML_["classes"]))
print(set(DF_A_res_ML_["subclass"]))

print(len(list(set(DF_A_res_ML_["Code"]))))
print(len(list(DF_A_res_ML_["Code"])))

{1, 2, 3, 4, 5, 6}
{'HN', 'NBL'}
{'Splenectomy', 'Normal', 'No_Splenectomy'}
51
51


In [68]:
def _CheckFoldML(DF_A_res_ML_):
    set_fold = list(set(DF_A_res_ML_["fold"]))
    for i in range(len(set_fold)):
        print(f"[INFO]: Fold-{set_fold[i]}")
        df_fold = DF_A_res_ML_[DF_A_res_ML_["fold"]==set_fold[i]]
        set_Ab = list(set(df_fold["subclass"]))
        set_Ab.sort()
        for j in range(len(set_Ab)):
            df_Ab_fold = df_fold[df_fold["subclass"]==set_Ab[j]]
            patian = list(set(df_Ab_fold["Code"]))
            print(f"      :{set_Ab[j]} ==> Number {len(patian)} Cases!!")
        print("="*125)

In [69]:
_CheckFoldML(DF_A_res_ML_)

[INFO]: Fold-1
      :No_Splenectomy ==> Number 3 Cases!!
      :Normal ==> Number 3 Cases!!
      :Splenectomy ==> Number 3 Cases!!
[INFO]: Fold-2
      :No_Splenectomy ==> Number 3 Cases!!
      :Normal ==> Number 2 Cases!!
      :Splenectomy ==> Number 3 Cases!!
[INFO]: Fold-3
      :No_Splenectomy ==> Number 3 Cases!!
      :Normal ==> Number 2 Cases!!
      :Splenectomy ==> Number 3 Cases!!
[INFO]: Fold-4
      :No_Splenectomy ==> Number 3 Cases!!
      :Normal ==> Number 3 Cases!!
      :Splenectomy ==> Number 3 Cases!!
[INFO]: Fold-5
      :No_Splenectomy ==> Number 3 Cases!!
      :Normal ==> Number 3 Cases!!
      :Splenectomy ==> Number 3 Cases!!
[INFO]: Fold-6
      :No_Splenectomy ==> Number 2 Cases!!
      :Normal ==> Number 3 Cases!!
      :Splenectomy ==> Number 3 Cases!!


In [72]:
DF_A_res_ML_ = DF_A_res_ML_.reset_index(drop=True)
DF_A_res_ML_.to_csv("/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_TrainML_6Fold_split2class.csv")
DF_A_res_ML_.head()

,Code,classes,subclass,fold,classes_binary,MCV,MCH,Hb
0,HN35,HN,No_Splenectomy,1,1.0,74.3,23.4,7.1
1,HN43,HN,No_Splenectomy,1,1.0,57.4,17.7,4.4
2,HN38,HN,No_Splenectomy,1,1.0,72.9,22.6,8.0
3,NBL14,NBL,Normal,1,0.0,86.8,28.1,13.2
4,NBL15,NBL,Normal,1,0.0,88.2,27.9,13.9


> ### 3 - Fold: 3 Classes       

In [74]:
print(set(DF_B_res["fold"]))
DF_B_res.head()

{1, 2, 3}


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,0.0,0.0,0.0,239.0,239.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
1,2.0,2.0,2.0,240.0,240.0,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
2,5.0,5.0,5.0,234.0,234.0,HN35_D0_30HZ_20XINF_UWELL_25660131_161754_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_161754,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
3,14.0,14.0,14.0,236.0,236.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162205_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162205,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
4,15.0,15.0,15.0,238.0,238.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162641_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162641,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1


In [78]:
#set(DF_B_res["Code"])-set(feture_ML["Code"])

In [79]:
#set(feture_ML["Code"])-set(DF_B_res["Code"])

In [80]:
DF_B_res_ML = DF_B_res.drop_duplicates(subset='Code', keep="last")
DF_B_res_ML_ = DF_B_res_ML[["Code", "classes", "subclass", "fold", "classes_binary", "MCV", "MCH", "Hb"]]
DF_B_res_ML_.reset_index(drop=True)
print(DF_B_res_ML_.shape)
DF_B_res_ML_

(51, 8)


,Code,classes,subclass,fold,classes_binary,MCV,MCH,Hb
4,HN35,HN,No_Splenectomy,1,1.0,74.3,23.4,7.1
12,HN38,HN,No_Splenectomy,1,1.0,72.9,22.6,8.0
15,HN43,HN,No_Splenectomy,1,1.0,57.4,17.7,4.4
20,NBL14,NBL,Normal,1,0.0,86.8,28.1,13.2
27,NBL23,NBL,Normal,1,0.0,66.5,20.3,13.0
31,NBL15,NBL,Normal,1,0.0,88.2,27.9,13.9
37,HN17,HN,Splenectomy,1,1.0,75.0,25.0,7.1
44,HN12,HN,Splenectomy,1,1.0,75.4,24.1,9.8
47,HN13,HN,Splenectomy,1,1.0,72.6,23.4,5.8
54,HN33,HN,No_Splenectomy,1,1.0,77.1,25.3,8.5


In [81]:
print(set(DF_B_res_ML_["fold"]))
print(set(DF_B_res_ML_["classes"]))
print(set(DF_B_res_ML_["subclass"]))

print(len(list(set(DF_B_res_ML_["Code"]))))
print(len(list(DF_B_res_ML_["Code"])))

{1, 2, 3}
{'HN', 'NBL'}
{'Splenectomy', 'Normal', 'No_Splenectomy'}
51
51


In [82]:
_CheckFoldML(DF_B_res_ML_)

[INFO]: Fold-1
      :No_Splenectomy ==> Number 6 Cases!!
      :Normal ==> Number 4 Cases!!
      :Splenectomy ==> Number 6 Cases!!
[INFO]: Fold-2
      :No_Splenectomy ==> Number 6 Cases!!
      :Normal ==> Number 6 Cases!!
      :Splenectomy ==> Number 6 Cases!!
[INFO]: Fold-3
      :No_Splenectomy ==> Number 5 Cases!!
      :Normal ==> Number 6 Cases!!
      :Splenectomy ==> Number 6 Cases!!


In [83]:
DF_B_res_ML_ = DF_B_res_ML_.reset_index(drop=True)
DF_B_res_ML_.to_csv("/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_TrainML_3Fold_split3class.csv")
DF_B_res_ML_.head()

,Code,classes,subclass,fold,classes_binary,MCV,MCH,Hb
0,HN35,HN,No_Splenectomy,1,1.0,74.3,23.4,7.1
1,HN38,HN,No_Splenectomy,1,1.0,72.9,22.6,8.0
2,HN43,HN,No_Splenectomy,1,1.0,57.4,17.7,4.4
3,NBL14,NBL,Normal,1,0.0,86.8,28.1,13.2
4,NBL23,NBL,Normal,1,0.0,66.5,20.3,13.0
